this script takes individual accessibility files that have been generated on euler and merges them into a single dataset + calculates differences for every person

In [1]:
import warnings
warnings.filterwarnings('ignore')

import geopandas
import r5py
import shapely
import pandas as pd
import geopandas as gpd
import numpy as np
import datetime
import copy

import os
import snman
from snman import osmnx_customized as oxc
from snman.constants import *

PERIMETER = '_accessibility_debug'

# Set these paths according to your own setup
data_directory = os.path.join('C:',os.sep,'Users','lballo','polybox','Research','SNMan','SNMan Shared','data_v2')
inputs_path = os.path.join(data_directory, 'inputs')
process_path = os.path.join(data_directory, 'process', PERIMETER)
outputs_path = os.path.join(data_directory, 'outputs', PERIMETER)

CRS_internal = 2056      # for Zurich

In [2]:
euler_files_path = os.path.join(outputs_path, 'euler')

In [3]:
def add_pseudo_person_id(accessibility):
    """
    add surrogate person id 
    """
    accessibility['record'] = accessibility.apply(
        lambda row: f"{row['statent_id']}_{row['age']}_{row['sex']}_{row['maritalstatus']}_{row['residencepermit']}_{row['geometry']}",
        axis=1
    )
    accessibility['record2'] = accessibility.groupby('record').cumcount()
    accessibility['record'] = accessibility['record'] + accessibility['record2'].astype(str)

def merge_for_state(state):
    """
    Merge all partial datasets into one
    """

    files = os.listdir(euler_files_path)
    files = list(
        filter(
            lambda item: (item[-5:] == '.gpkg') & (state in item),
            files
        )
    )
    datasets = map(
        lambda file: snman.io.import_geofile_to_gdf(
            str(os.path.join(euler_files_path, file))
        ).reset_index(),
        files
    )
    accessibility = gpd.GeoDataFrame(
        pd.concat(datasets)
    )
    accessibility['state'] = state
    #add_pseudo_person_id(accessibility)
    return accessibility
    
before = merge_for_state('before')
after = merge_for_state('after')

In [8]:
# merge before and after and calculate differences
data_fields = ['accessibility', 'accessibility_private_cars', 'accessibility_transit', 'accessibility_cycling', 'accessibility_foot']
m = pd.merge(before, after[data_fields + ['record']], on='record', suffixes=['_before', '_after'])

for field in data_fields:
    m[f'{field}_diff'] = m[f'{field}_after'] - m[f'{field}_before']
    m[f'{field}_diff_perc'] = (m[f'{field}_after'] - m[f'{field}_before']) / m[f'{field}_before']
    
    m[f'{field}_before_log'] = np.log(m[f'{field}_before'])
    m[f'{field}_after_log'] = np.log(m[f'{field}_after'])
    m[f'{field}_log_diff_perc'] = (m[f'{field}_after_log'] - m[f'{field}_before_log']) / m[f'{field}_before_log']
    

In [9]:




snman.io.export_gdf(
    m,
    os.path.join(outputs_path, 'accessibility_diff.gpkg')
) 

m

,index,record,age,sex,maritalstatus,residencepermit,residentpermit,statent_id,accessibility_before,accessibility_cycling_before,...,accessibility_cycling_diff,accessibility_cycling_diff_perc,accessibility_cycling_before_log,accessibility_cycling_after_log,accessibility_cycling_log_diff_perc,accessibility_foot_diff,accessibility_foot_diff_perc,accessibility_foot_before_log,accessibility_foot_after_log,accessibility_foot_log_diff_perc
0,0,7750768,38,1,1,301,3,68672447.0,69218.961169,50043.444111,...,12570.074185,0.251183,10.820647,11.044736,0.020709,-38.798105,-0.008234,8.457866,8.449598,-0.000978
1,1,9811619,48,2,2,-2,-2,68672447.0,69218.961169,50043.444111,...,12570.074185,0.251183,10.820647,11.044736,0.020709,-38.798105,-0.008234,8.457866,8.449598,-0.000978
2,2,11335928,56,1,2,201,2,68672447.0,69218.961169,50043.444111,...,12570.074185,0.251183,10.820647,11.044736,0.020709,-38.798105,-0.008234,8.457866,8.449598,-0.000978
3,3,537284,13,1,1,-2,-2,69702513.0,66234.221712,41129.292573,...,1345.187483,0.032706,10.624476,10.656659,0.003029,0.812481,0.000624,7.171789,7.172413,0.000087
4,4,3295332,20,2,1,-2,-2,69702513.0,66234.221712,41129.292573,...,1345.187483,0.032706,10.624476,10.656659,0.003029,0.812481,0.000624,7.171789,7.172413,0.000087
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40690,1179,4492458,30,2,1,-2,-2,68222474.0,93008.739384,100407.037620,...,13159.217982,0.131059,11.516988,11.640142,0.010693,417.121038,0.014398,10.274054,10.288349,0.001391
40691,1180,7631781,34,2,2,302,3,68222474.0,93008.739384,100407.037620,...,13159.217982,0.131059,11.516988,11.640142,0.010693,417.121038,0.014398,10.274054,10.288349,0.001391
40692,1181,9590269,0,2,1,-2,-2,68222474.0,93008.739384,100407.037620,...,13159.217982,0.131059,11.516988,11.640142,0.010693,417.121038,0.014398,10.274054,10.288349,0.001391
40693,1182,11214605,49,1,2,202,2,68222474.0,93008.739384,100407.037620,...,13159.217982,0.131059,11.516988,11.640142,0.010693,417.121038,0.014398,10.274054,10.288349,0.001391
